In [1]:
from eventregistry import *
import json, os, sys

from dotenv import load_dotenv
import os
from eventregistry import EventRegistry


load_dotenv()  # This loads the environment variables from .env file
api_key = os.getenv("EVENT_REGISTRY_API_KEY")
er = EventRegistry(apiKey=api_key, allowUseOfArchive=False)

In [2]:
keywords = "Iran's president"
q = QueryEventsIter(keywords=keywords)
for event in q.execQuery(er, sortBy="size", maxItems=1):
    print(json.dumps(event, indent=4))

{
    "uri": "spa-3486596",
    "concepts": [
        {
            "uri": "http://en.wikipedia.org/wiki/Iran",
            "type": "loc",
            "score": 100,
            "label": {
                "eng": "Iran"
            },
            "location": {
                "type": "country",
                "label": {
                    "eng": "Iran"
                }
            }
        },
        {
            "uri": "http://en.wikipedia.org/wiki/Helicopter",
            "type": "wiki",
            "score": 75,
            "label": {
                "eng": "Helicopter"
            }
        },
        {
            "uri": "http://en.wikipedia.org/wiki/President_of_Iran",
            "type": "wiki",
            "score": 61,
            "label": {
                "eng": "President of Iran"
            }
        },
        {
            "uri": "http://en.wikipedia.org/wiki/Azerbaijan",
            "type": "loc",
            "score": 33,
            "label": {
                "eng": 

In [3]:
eventUri = event["uri"]

In [28]:
lang = "zho"
q = QueryEventArticlesIter(eventUri=eventUri, lang=lang)
filename = "EA_" + eventUri + "_" + lang + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(
    er,
    sortBy="date",
    maxItems=10,
    returnInfo=ReturnInfo(
        sourceInfo=SourceInfoFlags(location=True),
    ),
):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

In [37]:
lang = "eng"
q = QueryEventArticlesIter(eventUri=eventUri, lang=lang)
filename = "EA_" + eventUri + "_" + lang + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(
    er,
    sortBy="date",
    maxItems=10,
    returnInfo=ReturnInfo(
        sourceInfo=SourceInfoFlags(location=True),
    ),
):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, indent=4)

In [7]:
from openai import OpenAI

load_dotenv(override=True)  # This loads the environment variables from .env file
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [35]:
def get_distinctive_events(
    art,
    client,
    model="gpt-4o",
):

    prompt_complete = (
        f"You are an expert at summarizing the news. \
            Read the following news article and translate it into English if necessary. \
                 Output a list of distinctive events described in the article. List the events in chronological order: "
        + art["body"]
    )
    model = model
    messages = [{"role": "user", "content": prompt_complete}]
    try:
        response = client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )

        content = response.choices[0].message.content
        return content
    except Exception as e:  # if the model fails to return a response
        print(f"Error: {e}")
        return "Sorry, error from GPT."

In [36]:
src_filename = "EA_" + eventUri + "_" + lang + ".json"
with open(src_filename, "r") as f:
    data = f.read()
    des_articles = []
    src_articles = json.loads(data)
    for src_art in src_articles:
        src_art["distinctive_events"] = get_distinctive_events(
            src_art, client, model="gpt-4o"
        )

        des_articles.append(src_art)

des_filename = "DE_" + src_filename
if os.path.exists(des_filename):
    os.remove(des_filename)
with open(des_filename, "w") as f:
    json.dump(des_articles, f, indent=4)

In [30]:
lang = "zho"
src_filename = "EA_" + eventUri + "_" + lang + ".json"
with open(src_filename, "r") as f:
    data = f.read()
    des_articles = []
    src_articles = json.loads(data)
    for src_art in src_articles:
        src_art["distinctive_events"] = get_distinctive_events(
            src_art, client, model="gpt-4o"
        )

        des_articles.append(src_art)

des_filename = "DE_" + src_filename
if os.path.exists(des_filename):
    os.remove(des_filename)
with open(des_filename, "w") as f:
    json.dump(des_articles, f, ensure_ascii=False, indent=4)

In [31]:
print(des_articles[0]["distinctive_events"])

1. **Helicopter Accident**: On the 19th, a helicopter carrying Iranian President Ebrahim Raisi and other officials made a hard landing within Iran.

2. **Initial Communication Attempts**: On the 21st, the head of the Iranian President's office, Hossein, stated that after the accident, the office tried to call the President and the Foreign Minister's mobile phones, but no one answered.

3. **Contact with Survivor**: They then called the pilot's phone, and another person on the helicopter, Al-Hashim, answered. He reported that he had fallen onto a mountain, felt very unwell, and did not know his location or the whereabouts of the others.

4. **Confirmation of Accident**: After the phone call, the President's office confirmed that the helicopter had indeed been in an accident and had multiple conversations with Al-Hashim.

5. **Casualty Confirmation**: Hossein mentioned that after rescue personnel found the bodies of the other victims, it was determined that they had died immediately foll

In [50]:
lang = "zho"
src_filename = "DE_EA_" + eventUri + "_" + lang + ".json"
with open(src_filename, "r") as f:
    data = f.read()

    src_articles = json.loads(data)
    print(src_articles[2]["distinctive_events"])

Here is a list of distinctive events described in the article, in chronological order:

1. **Iranian President's Helicopter Incident**: The Iranian President's helicopter, a Bell 212 model manufactured by the United States, experienced a "hard landing" recently.

2. **Iranian President's Death**: As a result of the "hard landing," the Iranian President passed away.

3. **Historical Context**: The article provides historical context, mentioning that during the Pahlavi era, Iran and the United States had a good relationship, with the U.S. selling advanced military equipment to Iran, including the Bell 212 helicopter and F-14 fighter jets.

4. **Blame on U.S. Sanctions**: The article discusses the viewpoint that U.S. sanctions and long-term embargoes on Iran led to a lack of spare parts and proper maintenance for the helicopter, contributing to the incident.

5. **Iran's Request for U.S. Assistance**: After the helicopter incident, Iran sought help from the United States. Initially, the U

In [17]:
load_dotenv()  # This loads the environment variables from .env file
api_key = os.getenv("EVENT_REGISTRY_API_KEY")
er = EventRegistry(apiKey=api_key, allowUseOfArchive=False)
q = GetEventForText(er)
for event in q.compute(lang="eng", text=des_articles[0]["distinctive_events"]):
    print(json.dumps(event, indent=4))

{
    "storyUri": "eng-9581242",
    "eventUri": "zho-1619666",
    "storyInfo": {
        "uri": "eng-9581242",
        "articleCount": 3114,
        "lang": "eng",
        "avgCos": 0.5450980392156862,
        "variance": 0.7058823529411765,
        "eventUri": "zho-1619666",
        "title": "Live updates: Iranian President Raisi involved in helicopter crash",
        "isUserSetTitle": false,
        "summary": "From CNN's Artemis Moshtaghian, Nechirvan Mando, Rosa Rahimi and Eve Brennan\n\nTwenty rescue teams and drones have been sent to the area where a helicopter carrying Iranian President Ebrahim Raisi and Foreign Minister Hossein Amir-Abdollahian crashed on Sunday, according to Iranian state news agency IRNA.\n\nThe pair, along with a group of dignitaries, were aboard the helicopter returning from a ceremony for an opening of a dam on Iran's border with Azerbaijan when the aircraft \"crashed upon landing",
        "isUserSetSummary": false,
        "concepts": [
            {
 